In [1]:
import hopsworks

In [2]:
from utils.feature_engineering import get_latest_sold_products
get_latest_sold_products()[0]

{'badges': [],
 'bumped_at': '2025-01-03T13:29:07.940Z',
 'buynow': True,
 'category_path_size': 'footwear.lowtop_sneakers.10',
 'category_path': 'footwear.lowtop_sneakers',
 'category_size': 'footwear.10',
 'category': 'footwear',
 'color': 'black',
 'condition': 'is_new',
 'cover_photo': {'id': 513226876,
  'listing_id': 71590415,
  'image': None,
  'created_at': '2024-08-20T10:21:16.176Z',
  'updated_at': '2024-12-06T13:28:58.678Z',
  'photoable_id': 71590415,
  'photoable_type': 'Listing',
  'width': 1200,
  'height': 1600,
  'url': 'https://media-assets.grailed.com/prd/listing/temp/13c3a8cc3bff426b96c3c7275174fbd8',
  'position': 0,
  'rotate': 0,
  'image_url': 'https://media-assets.grailed.com/prd/listing/temp/13c3a8cc3bff426b96c3c7275174fbd8'},
 'created_at_i': 1733491738,
 'created_at': '2024-12-06T13:28:58.738Z',
 'deleted': False,
 'department': 'menswear',
 'description': 'Brand new, never worn and comes the original shoebox and everything that comes with it\n\n100% authent

In [3]:
from dotenv import load_dotenv
load_dotenv()
    
project = hopsworks.login()

2025-01-07 16:23:30,902 INFO: Initializing external client
2025-01-07 16:23:30,904 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 16:23:32,809 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159324


## Get data

### Quick data exploration
Taken from https://graildient-descent.streamlit.app/eda, using 10,000 samples.
We could do the same using the larger dataset.

#### Numerical/quantitative features
Target variable (sold_price):
- Most sold items are between 35-135$ (consider plotting bins) - consider outliers, since we are far from normal distribution, maybe try a log transformation?

Number of photos:
- Another numerical feature, could be added
- Price increases until 13 photos, then inconsistent

#### Categorical/qualitative features
- high-cardinality: designer, color, subcategory, size
- low-cardinality: category, condition

Target encoding more fitting for high-cardinality features. Whereas low-cardinality features could be one-hot encoded.

- designer a strong predictor
- we skip department, to focus on men's clothing only (it has better representation)
- there is good variation of sold prices in different subcategories => probably a good indicator
- could be interesting to use embeddings for color instead
- condition - perfect for ordinal encoding. The better the condition, the higher the average sold price.

##### Text
- title/description - should we do any pre-processing?
- we could look at sentiment and similar text analysis approaches

#### Images
- could be interesting to add embedding representation of title image

## Feature processing

In [4]:
from utils.feature_engineering import pipeline
df = pipeline(no_of_hits=2000)
df.shape

embedding designer names
embedding descriptions
embedding titles
embedding hashtags
embedding size
shape: (1_000, 12)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id       ┆ sold_at   ┆ designer_ ┆ descripti ┆ … ┆ size      ┆ color     ┆ followern ┆ sold_pric │
│ ---      ┆ ---       ┆ names     ┆ on        ┆   ┆ ---       ┆ ---       ┆ o         ┆ e         │
│ i64      ┆ datetime[ ┆ ---       ┆ ---       ┆   ┆ list[f32] ┆ i64       ┆ ---       ┆ ---       │
│          ┆ μs]       ┆ list[f32] ┆ list[f32] ┆   ┆           ┆           ┆ i64       ┆ i64       │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 71590415 ┆ 2025-01-0 ┆ [0.007445 ┆ [-0.01228 ┆ … ┆ [-0.04139 ┆ 418332173 ┆ 16        ┆ 1259      │
│          ┆ 7 15:22:4 ┆ ,         ┆ 6,        ┆   ┆ , -0.0250 ┆ 0         ┆           ┆           │
│          ┆ 0.967     ┆ 0.028137, ┆ 0.000737, ┆   ┆ 74, …     ┆          

(1000, 12)

## Save data

In [5]:
fs = project.get_feature_store() 

In [6]:
grailed_items_fg = fs.get_or_create_feature_group(
    name='draft_grailed_items',
    description='Sold Grailed items',
    version=4,
    primary_key=['id'],
    event_time="sold_at",
    # expectation_suite=aq_expectation_suite
)

In [7]:
grailed_items_fg.insert(df)

Uploading Dataframe: 100.00% |███████████████████████| Rows 1000/1000 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: draft_grailed_items_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1159324/jobs/named/draft_grailed_items_4_offline_fg_materialization/executions


(Job('draft_grailed_items_4_offline_fg_materialization', 'SPARK'), None)

In [8]:
# TODO: Update feature description